In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

In [2]:
Rootdf = pd.read_csv('Root_Insurance_data_Edited.csv')


Rootdf['Multi_VD'] = Rootdf.Num_Vehicles * Rootdf.Num_Drivers
Rootdf['Add_VD'] = Rootdf.Num_Vehicles + Rootdf.Num_Drivers

Rootdf = Rootdf.replace('Y',1)
Rootdf = Rootdf.replace('N',0)
Rootdf = Rootdf.replace('U',2)
Rootdf = Rootdf.replace('S',0)
Rootdf = Rootdf.replace('M',1)


#Rootdf.Insured[Rootdf.Insured == 'N'] = 0
#Rootdf.Insured[Rootdf.Insured == 'Y'] = 1
#Rootdf.Insured[Rootdf.Insured == 'U'] = 2

#Rootdf.Marital[Rootdf.Marital == 'S'] = 0
#Rootdf.Marital[Rootdf.Marital == 'M'] = 1

#Rootdf.Click[Rootdf.Click == 'N'] = 0
#Rootdf.Click[Rootdf.Click == 'Y'] = 1

In [3]:
Rootdf.head()

,Insured,Num_Vehicles,Num_Drivers,Marital,Bid,Rank,Click,Policies_sold,Multi_VD,Add_VD
0,2,2,1,1,10,2,0,0,2,3
1,1,3,1,1,10,5,0,0,3,4
2,1,3,2,1,10,5,0,0,6,5
3,0,3,2,0,10,4,0,0,6,5
4,2,2,2,0,10,2,0,0,4,4


In [4]:
X = Rootdf[["Insured", "Num_Vehicles", "Num_Drivers", "Marital","Multi_VD","Add_VD"]]
Y = Rootdf.Rank
X.head()

,Insured,Num_Vehicles,Num_Drivers,Marital,Multi_VD,Add_VD
0,2,2,1,1,2,3
1,1,3,1,1,3,4
2,1,3,2,1,6,5
3,0,3,2,0,6,5
4,2,2,2,0,4,4


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,
                                                    random_state = 314,
                                                    shuffle=True,
                                                    stratify=Rootdf[['Rank','Click','Policies_sold']])

In [6]:
#print(x_train.Rank.value_counts()/x_train.Rank.count())
#print(x_test.Rank.value_counts()/x_test.Rank.count())

#print(x_train.Click.value_counts()/x_train.Click.count())
#print(x_test.Click.value_counts()/x_test.Click.count())

print(y_train.value_counts()/y_train.count())
print(y_test.value_counts()/y_test.count())

3    0.240125
5    0.229125
4    0.208875
1    0.161000
2    0.160875
Name: Rank, dtype: float64
3    0.2400
5    0.2290
4    0.2090
1    0.1615
2    0.1605
Name: Rank, dtype: float64


In [7]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

def powerset(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
        
    return power_set[1:]

In [8]:
## get all models we're interested in
model = ["Insured", "Num_Vehicles", "Num_Drivers"]

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 5309)

## make empty mse holder
cv_roc_auc_ovo = np.zeros((5, 2))
cv_roc_auc_ovr = np.zeros((5, 2))
cv_acc_score = np.zeros((5, 2))


## loop through all splits
i = 0
for train_index, test_index in skfold.split(x_train,y_train):
    ## get train and holdout sets
    x_train_train = x_train.iloc[train_index]
    x_holdout = x_train.iloc[test_index]
    y_train_train = y_train.iloc[train_index]
    y_holdout = y_train.iloc[test_index]
    
    ## make clone
    for j in range(0,2):
        forest = RandomForestClassifier(n_estimators = 50+100*j, max_depth = 6, random_state = 212)
        forest_clone = clone(forest)
        
        ## fit clone
        forest_clone.fit(x_train_train[model],y_train_train)

        y_proba_predict = forest_clone.predict_proba(x_holdout[model])

        ## record mse
        cv_roc_auc_ovo[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovo')
        cv_roc_auc_ovr[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovr')
        cv_acc_score[i,j] = accuracy_score(y_holdout,forest_clone.predict(x_holdout[model]))
    i=i+1  

In [12]:
for i in range(0,2):
    print("For Model ",model, "\nNumber of Estimators ",50+100*i,
          "\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo,axis=0)[i],
          "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr,axis=0)[i],
          "\nThe Accuracy score is ",np.mean(cv_acc_score,axis=0)[i],"\n")

For Model  ['Insured', 'Num_Vehicles', 'Num_Drivers'] 
Number of Estimators  50 
The OVO ROC score is  0.7544837106966362 
The OVR ROC score is  0.7535786285495851 
The Accuracy score is  0.4285 

For Model  ['Insured', 'Num_Vehicles', 'Num_Drivers'] 
Number of Estimators  150 
The OVO ROC score is  0.7544094293944374 
The OVR ROC score is  0.7535604884594262 
The Accuracy score is  0.43 



In [15]:
forest1 = RandomForestClassifier(n_estimators = 50, max_depth = 6, random_state = 212)

forest1.fit(x_train[model],y_train)
y_proba_predict = forest1.predict_proba(x_test[model])

cv_roc_auc_ovo1 = roc_auc_score(y_test,y_proba_predict,multi_class='ovo')
cv_roc_auc_ovr1 = roc_auc_score(y_test,y_proba_predict,multi_class='ovr')
cv_acc_score1 = accuracy_score(y_test,forest1.predict(x_test[model]))

print("For Model ",model, "\nNumber of Estimators ",50,
      "\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo,axis=0)[i],
      "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr,axis=0)[i],
      "\nThe Accuracy score is ",np.mean(cv_acc_score,axis=0)[i],"\n")

forest2 = RandomForestClassifier(n_estimators = 150, max_depth = 6, random_state = 212)

forest2.fit(x_train[model],y_train)
y_proba_predict = forest2.predict_proba(x_test[model])

cv_roc_auc_ovo2 = roc_auc_score(y_test,y_proba_predict,multi_class='ovo')
cv_roc_auc_ovr2 = roc_auc_score(y_test,y_proba_predict,multi_class='ovr')
cv_acc_score2 = accuracy_score(y_test,forest2.predict(x_test[model]))
    
print("For Model ",model, "\nNumber of Estimators ",150,
      "\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo,axis=0)[i],
      "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr,axis=0)[i],
      "\nThe Accuracy score is ",np.mean(cv_acc_score,axis=0)[i],"\n")

For Model  ['Insured', 'Num_Vehicles', 'Num_Drivers'] 
Number of Estimators  50 
The OVO ROC score is  0.7544094293944374 
The OVR ROC score is  0.7535604884594262 
The Accuracy score is  0.43 

For Model  ['Insured', 'Num_Vehicles', 'Num_Drivers'] 
Number of Estimators  150 
The OVO ROC score is  0.7544094293944374 
The OVR ROC score is  0.7535604884594262 
The Accuracy score is  0.43 



In [124]:
score1_df = pd.DataFrame({'feature':x_test[model].columns,
                            'importance_score': forest1.feature_importances_})

score1_df.sort_values('importance_score',ascending=False)

,feature,importance_score
0,Insured,0.626741
1,Num_Vehicles,0.336341
2,Num_Drivers,0.036918


In [125]:
score2_df = pd.DataFrame({'feature':x_test[model].columns,
                            'importance_score': forest2.feature_importances_})

score2_df.sort_values('importance_score',ascending=False)

,feature,importance_score
0,Insured,0.631721
1,Num_Vehicles,0.334031
2,Num_Drivers,0.034247


In [120]:
Insured = [0,1,2]
Num_Vehicles = [1,2,3]
Num_Drivers = [1,2]
Ranks = [1,2,3,4,5]
Model_Rank=[]
for x in Insured:
    for y in Num_Vehicles:
        for z in Num_Drivers:
            insert = {"Insured":x,"Num_Vehicles":y,"Num_Drivers":z}
            model_df = {"Insured":[x],"Num_Vehicles":[y],"Num_Drivers":[z]}
            temp_df = pd.DataFrame.from_dict(model_df)
            predict_proba = forest1.predict_proba(temp_df)
            insert["Expected Rank"] = np.sum(predict_proba[0]*Ranks)
            insert["Most Prob Rank"] = Ranks[np.argmax(predict_proba[0])]
            insert["Rank Prob Skew"] = st.skew(predict_proba[0]*[-2,-1,0,1,2])
            for i in range(0,5):
                insert["Rank "+str(i+1)+" Prob"] = predict_proba[0,i]
            #insert[""]
            Model_Rank.append(insert)            

In [121]:
Model_Rank_df = pd.DataFrame(Model_Rank)
Model_Rank_df.Insured[Model_Rank_df.Insured == 0] = 'N'
Model_Rank_df.Insured[Model_Rank_df.Insured == 1] = 'Y'
Model_Rank_df.Insured[Model_Rank_df.Insured == 2] = 'U'
Model_Rank_df

<ipython-input-121-2dad277a43e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Model_Rank_df.Insured[Model_Rank_df.Insured == 0] = 'N'


,Insured,Num_Vehicles,Num_Drivers,Expected Rank,Most Prob Rank,Rank Prob Skew,Rank 1 Prob,Rank 2 Prob,Rank 3 Prob,Rank 4 Prob,Rank 5 Prob
0,N,1,1,3.528295,3,0.178298,0.000000,0.187059,0.304863,0.300804,0.207275
1,N,1,2,2.427277,3,-0.392843,0.235256,0.271700,0.323555,0.169489,0.000000
2,N,2,1,4.318991,5,1.303468,0.000000,0.000000,0.212065,0.256880,0.531055
3,N,2,2,3.543329,4,0.154298,0.000000,0.172030,0.306328,0.327925,0.193717
4,N,3,1,4.292866,5,1.127976,0.000000,0.000000,0.187573,0.331988,0.480439
5,N,3,2,4.251878,5,1.185415,0.000000,0.000000,0.224670,0.298781,0.476548
6,Y,1,1,3.060226,3,0.049581,0.104411,0.219311,0.302851,0.258496,0.114932
7,Y,1,2,2.510178,3,-0.100225,0.186765,0.303180,0.323166,0.186889,0.000000
8,Y,2,1,4.293832,5,1.249579,0.000000,0.000000,0.213201,0.279767,0.507032
9,Y,2,2,4.329466,5,1.183896,0.000000,0.000000,0.176285,0.317964,0.505751


In [116]:
st.skew(np.array([0,0,0.2,0.3,0.1])*[-2,-1,0,1,2])

0.5929270612815707

In [105]:
predict_proba = forest1.predict_proba(temp_df)
type(predict_proba)

numpy.ndarray